In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from tqdm import tqdm
from pandas.tseries.offsets import BDay
import re
import os
from dateutil.relativedelta import relativedelta
from opts import FuturesCurve

import warnings
warnings.filterwarnings("ignore")


List of all Settlements archives

In [2]:
lst =os.listdir('data/Settlements')
lst[0:10]

['Corn_19_1_2018_ags_settlements.txt',
 'Wheat_2_6_2017_ags_settlements.txt',
 '13_11_2017_ags_settlements.txt',
 'Soybean Oil_26_9_2017_ags_settlements.txt',
 '21_8_2018_eonly_settlements.txt',
 'Soybean Oil_24_6_2019_ags_settlements.txt',
 'Wheat_17_4_2019_ags_settlements.txt',
 '7_11_2016_ags_settlements.txt',
 '16_7_2018_ags_settlements.txt',
 'Soybeans_13_12_2016_ags_settlements.txt']

Getting first entry in file name to identify agricultural contacts

In [3]:
archs = pd.DataFrame([l.split('_') for l in lst])
list(set(archs[0]))[0:10]

['7',
 '16',
 'soybean meal',
 '30',
 'Soybeans',
 '21',
 '18',
 'soybean oil',
 '13',
 'Wheat']

Identifying agg contracts and separating by asset, sorting by date

In [4]:
archs_corn = archs[ (archs[0]=='Corn') | (archs[0]=='corn') ]
archs_wheat = archs[ (archs[0]=='Wheat') | (archs[0]=='wheat') ]
archs_soybeans = archs[ (archs[0]=='Soybeans') | (archs[0]=='soybeans') ]

archs_corn[[1,2,3]] = archs_corn[[1,2,3]].astype('int')
archs_wheat[[1,2,3]] = archs_wheat[[1,2,3]].astype('int')
archs_soybeans[[1,2,3]] = archs_soybeans[[1,2,3]].astype('int')

arches_corn = archs_corn.sort_values(by=[3,2,1])
arches_wheat = archs_wheat.sort_values(by=[3,2,1])
arches_soybeans = archs_soybeans.sort_values(by=[3,2,1])

print("# of contracts corn: ",arches_corn.shape[0])
print("# of contracts wheat: ",arches_wheat.shape[0])
print("# of contracts soybeans: ",arches_soybeans.shape[0])

#first Corn contract
lst[arches_corn.index[0]]

# of contracts corn:  457
# of contracts wheat:  424
# of contracts soybeans:  440


'corn_21_9_2016_settlements.txt'

Creating a dict with keys the months in the Settlements file format, and values as contract expirations in datetime format. Agricultural products expire on the 14th of every month, or the previous business day if a weekend/holiday

In [5]:
mon = ['JAN','FEB',"MAR",'APR','MAY','JUN','JLY','AUG','SEP','OCT','NOV','DEC']
dummy_dict = {i : mon[i-1] for i in range(1,13)}
exp_dict = {}
for year in [16,17,18,19,20]:
    for month in range(1,13):
        exp_dict[mon[month - 1]+str(year)] = datetime(2000 + year, month, 15) - BDay(1)

exp_dict['JAN16']

Timestamp('2016-01-14 00:00:00')

In [6]:
def scrape_settlements(asset):
    
    # Futures curve for futures prices under 2 months in case settlement file doesn't 
    # have futures contract for an options expiry 
    ft = FuturesCurve()
    asset_full_dict = {}
    
    # Indices for a specific asset's settlement files
    if asset == 'Corn':
        arches = arches_corn
    elif asset == 'Wheat':
        arches = arches_wheat
    elif asset == 'Soybeans':
        arches = arches_soybeans
        
    # List with dates from each settlement file name in datetime
    dates_list = []
    for ind in arches.index:
        dates_list.append(datetime(int(arches[3].loc[ind]), 
                               int(arches[2].loc[ind]), int(arches[1].loc[ind])))    
    
    # Loop through all files
    for arch_ind, arch in enumerate(tqdm(arches.index)):
        
        # If there's any unidentified error, for a particular date (could be corrupt file or other)
        # ignore that date
        try:
        
            date_dict = {}
            doc_date = dates_list[arch_ind]
            doc = []
            with open('data/Settlements/' + lst[arch]) as f:
                for row in f:
                    doc.append(row) 

            #Identifying options contracts by code
            if asset == 'Corn':
                cont_name = 'PY'
            elif asset == 'Wheat':
                cont_name = 'WZ'
            elif asset == "Soybeans":
                cont_name = 'CZO'

            # Finding indices inside the document for each option expiration for lead contracts
            py = [d.startswith(cont_name) for d in doc] 
            py_index = pd.DataFrame(doc)[py].index

            # Putting document in a DataFrame
            text = [re.sub("\s+", ",", d.strip()).split(',') for d in doc]
            df = pd.DataFrame(text)
            df = df[[0,1,2,3,4,8,10]]
            df['expiration'] = np.zeros(df.shape[0])
            df['future'] = np.zeros(df.shape[0])
            df['date'] = dates_list[arch_ind]

            # Converting column 8 (yesterday's settlement prices) into a float with same format as strike column, 
            # or empty string if non numerical data
            for ind in df.index:
                num = str(df[[8]].iloc[ind]).split()[1].split('\'')
                try:
                    if num[0] == '':
                        num[0] = 0
                    num = 10*float(num[0]) + float(num[1])
                    df[8].loc[ind] = num
                except:
                    pass

            # Identifying futures prices for the day
            futs = df[[0,8]][1:6]
            futs = futs.set_index(0)

            # Identifying if call or put
            py_index_call = py_index[df.iloc[py_index][4] == 'CALL']
            py_index_put = py_index[df.iloc[py_index][4] == 'PUT']

            # Looping through both calls and puts for a specific options code
            for py_i, py_ind in enumerate([py_index_call, py_index_put]):
                vol_dict = {}

                # Focus only on the first 5 expirations date for each contract
                for i in range(5):
                    last_ind = py_ind[i]
                    exp = str(df.iloc[py_ind[i]][1])

                    # Only look at options with at least 7 business days to expiration to avoid
                    # implied volatility calculation issues
                    if exp_dict[exp] >= doc_date + BDay(7):

                        # Check if we have a futures price for a particular options expiration
                        if exp in futs.index:
                            while df[0][last_ind + 1].isnumeric():
                                last_ind += 1 
                            df.expiration[py_index[i]:last_ind] = exp_dict[exp]
                            df.future = float(futs.loc[exp])
                            vol_dict[i] = df[py_ind[i]:last_ind][1:].dropna()
                            vol_dict[i] = vol_dict[i][[0,8,10,'expiration','future','date']]
                            vol_dict[i] = vol_dict[i].rename(columns={0:'strike',8:'settle',10:'oi'})
                            vol_dict[i].strike = vol_dict[i].strike.astype('float')
                            vol_dict[i].settle = vol_dict[i].settle.astype('float')
                            vol_dict[i].oi = vol_dict[i].oi.astype('int')

                        # If we don't have the futures price for a particular option, get it from
                        # FuturesCurve by interpolation if under two months
                        elif exp_dict[exp] <= pd.to_datetime(doc_date + relativedelta(months = 2)- BDay(2)):
                            if asset == 'Corn':
                                fut_code = 'C'
                            elif asset == 'Wheat':
                                fut_code = 'W'
                            else:
                                fut_code = 'S'

                            # If there's an key error retreiving an interpolated futures price, 
                            # ignore options contract
                            try:
                                # Get price from FuturesCurve
                                fut_px = ft.get(fut_code, doc_date - BDay(1), exp_dict[exp])
                                while df[0][last_ind + 1].isnumeric():
                                    last_ind += 1 
                                df.expiration[py_index[i]:last_ind] = exp_dict[exp]
                                # 10 * adjustment to match strike format
                                df.future = 10*fut_px
                                vol_dict[i] = df[py_ind[i]:last_ind][1:].dropna()
                                vol_dict[i] = vol_dict[i][[0,8,10,'expiration','future','date']]
                                vol_dict[i] = vol_dict[i].rename(columns={0:'strike',8:'settle',10:'oi'})
                                vol_dict[i].strike = vol_dict[i].strike.astype('float')
                                vol_dict[i].settle = vol_dict[i].settle.astype('float')
                                vol_dict[i].oi = vol_dict[i].oi.astype('int')
                            except KeyError:
                                vol_dict[i] = ''
                        else:
                            vol_dict[i] = ''
                    # In any other case, return empty string for an options contract
                    else:
                        vol_dict[i] = ''

                if py_i == 0:
                    date_dict['Call'] = vol_dict
                else:
                    date_dict['Put'] = vol_dict
            asset_full_dict[doc_date] = date_dict
        except:
            print('Error with archive : {} in arches.index', arch)

    return asset_full_dict



In [7]:
corn_dict = scrape_settlements('Corn')

100%|██████████| 457/457 [22:01<00:00,  2.89s/it]


In [16]:
wheat_dict = scrape_settlements('Wheat')

 87%|████████▋ | 368/424 [19:29<01:41,  1.81s/it]

Error with archive : {} in arches.index 1504
Error with archive : {} in arches.index 727


100%|██████████| 424/424 [21:00<00:00,  2.97s/it]


In [17]:
soybeans_dict = scrape_settlements('Soybeans')

 87%|████████▋ | 382/440 [19:38<02:22,  2.45s/it]

Error with archive : {} in arches.index 951
Error with archive : {} in arches.index 1698


100%|██████████| 440/440 [21:42<00:00,  2.96s/it]


In [18]:
corn_dict[list(corn_dict.keys())[50]]['Call'][2]

,strike,settle,oi,expiration,future,date
1176,3400.0,247.0,7846,2017-03-14 00:00:00,3604.0,2016-12-13
1177,3450.0,212.0,448,2017-03-14 00:00:00,3604.0,2016-12-13
1178,3500.0,177.0,26013,2017-03-14 00:00:00,3604.0,2016-12-13
1179,3550.0,147.0,1005,2017-03-14 00:00:00,3604.0,2016-12-13
1180,3600.0,122.0,18795,2017-03-14 00:00:00,3604.0,2016-12-13
1181,3650.0,100.0,683,2017-03-14 00:00:00,3604.0,2016-12-13
1182,3700.0,80.0,14994,2017-03-14 00:00:00,3604.0,2016-12-13
1183,3750.0,63.0,2116,2017-03-14 00:00:00,3604.0,2016-12-13
1184,3800.0,51.0,17392,2017-03-14 00:00:00,3604.0,2016-12-13
1185,3850.0,41.0,1093,2017-03-14 00:00:00,3604.0,2016-12-13


In [19]:
import pickle

with open('data/corn_dict.pickle', 'wb') as handle:
    pickle.dump(corn_dict, handle)
with open('data/wheat_dict.pickle', 'wb') as handle:
    pickle.dump(wheat_dict, handle)
with open('data/soybeans_dict.pickle', 'wb') as handle:
    pickle.dump(soybeans_dict, handle)